In [ ]:
from bs4 import BeautifulSoup as BS
import requests
import re

In [ ]:
with open('index.html', 'r') as f:
    doc = BS(f, 'html.parser')

In [ ]:
print(doc.prettify())

In [ ]:
##To find/access tag name

tag = doc.title

print(tag) #it prints just the first tag
print(tag.find('a')) #to access the first tag that contains 'a'
print(tag.find_all('a')) #to acces all the tag that contains 'a'
print(tag.string) #to print the string inside the tag
print(tag.find_all('a')[1]) #to acces the tag inside tag 'a'

tag.string ='hello' #to change what is inside the tag

In [ ]:
tag

In [ ]:
url  = 'https://www.kijiji.ca/b-apartments-condos/ontario/for-rent/basement+apartment/page-2/k0c37l9004a29276001'
result = requests.get(url)
print(result.text)


In [ ]:
doc = BS(result.text, 'html.parser')
print(doc.prettify())

In [ ]:
prices =doc.find_all(text = '$')
print(prices)


In [ ]:
##if you are not able to access the exact details you want then access the parent tag of the tag that contains the information you need

parent = prices[0].parent
print(parent)

##to access the actual detail you need then access the strong tag within it
strong =parent.find('strong')
print(strong)

## to retrieve the text within it
print(strong.string)

In [ ]:
search_term = input("What product do you want to search for? ")

url = f"https://www.newegg.ca/p/pl?d={search_term}&N=4131"
page = requests.get(url).text
doc = BeautifulSoup(page, "html.parser")

In [ ]:
page_text = doc.find(class_="list-tool-pagination-text").strong
pages = int(str(page_text).split("/")[-2].split(">")[-1][:-1])

items_found = {}

In [ ]:


for page in range(1, pages + 1):
    url = f"https://www.newegg.ca/p/pl?d={search_term}&N=4131&page={page}"
    page = requests.get(url).text
    doc = BeautifulSoup(page, "html.parser")
    
    div = doc.find(class_="item-cells-wrap border-cells items-grid-view four-cells expulsion-one-cell")
    items = div.find_all(text=re.compile(search_term))

    for item in items:
        parent = item.parent
        if parent.name != "a":
            continue

        link = parent['href']
        next_parent = item.find_parent(class_="item-container")
        try:
            price = next_parent.find(class_="price-current").find("strong").string
            items_found[item] = {"price": int(price.replace(",", "")), "link": link}
        except:
            pass

sorted_items = sorted(items_found.items(), key=lambda x: x[1]['price'])

for item in sorted_items:
    print(item[0])
    print(f"${item[1]['price']}")
    print(item[1]['link'])
    print("-------------------------------")
